In [6]:
!pip install yt_dlp moviepy pydub
!apt-get install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [7]:
%%writefile 102313059.py
import sys
import os
import yt_dlp
from moviepy.editor import VideoFileClip
from pydub import AudioSegment

def download_videos(singer_name, number_of_videos):
    print("Searching and downloading videos...")

    ydl_opts = {
        'format': 'mp4',
        'outtmpl': 'videos/%(title)s.%(ext)s',
        'quiet': False,
        'noplaylist': True
    }

    os.makedirs("videos", exist_ok=True)

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            search_query = f"ytsearch{number_of_videos}:{singer_name} songs"
            ydl.download([search_query])
    except Exception as e:
        print("Error while downloading videos:", e)
        sys.exit(1)


def convert_videos_to_audio():
    print("Converting videos to audio...")
    os.makedirs("audios", exist_ok=True)

    for file in os.listdir("videos"):
        if file.endswith(".mp4"):
            video_path = os.path.join("videos", file)
            audio_path = os.path.join("audios", file.replace(".mp4", ".mp3"))

            try:
                video = VideoFileClip(video_path)
                video.audio.write_audiofile(audio_path)
                video.close()
            except Exception as e:
                print("Error converting:", file, e)


def cut_audio(duration):
    print(f"Cutting first {duration} seconds from each audio...")
    os.makedirs("trimmed", exist_ok=True)

    for file in os.listdir("audios"):
        if file.endswith(".mp3"):
            audio_path = os.path.join("audios", file)
            trimmed_path = os.path.join("trimmed", file)

            try:
                audio = AudioSegment.from_mp3(audio_path)
                trimmed_audio = audio[:duration * 1000]
                trimmed_audio.export(trimmed_path, format="mp3")
            except Exception as e:
                print("Error trimming:", file, e)


def merge_audios(output_filename):
    print("Merging all trimmed audios...")
    combined = AudioSegment.empty()

    for file in os.listdir("trimmed"):
        if file.endswith(".mp3"):
            file_path = os.path.join("trimmed", file)
            try:
                audio = AudioSegment.from_mp3(file_path)
                combined += audio
            except Exception as e:
                print("Error merging:", file, e)

    try:
        combined.export(output_filename, format="mp3")
        print("Mashup created successfully:", output_filename)
    except Exception as e:
        print("Error exporting final file:", e)


def main():
    # DEBUG: show arguments
    print("Arguments received:", sys.argv)

    if len(sys.argv) != 5:
        print("Usage: python <program.py> <SingerName> <NumberOfVideos> <AudioDuration> <OutputFileName>")
        sys.exit(1)

    singer_name = sys.argv[1]

    try:
        number_of_videos = int(sys.argv[2])
        duration = int(sys.argv[3])
    except ValueError:
        print("NumberOfVideos and AudioDuration must be integers.")
        sys.exit(1)

    output_filename = sys.argv[4]

    if number_of_videos <= 10:
        print("NumberOfVideos must be greater than 10.")
        sys.exit(1)

    if duration <= 20:
        print("AudioDuration must be greater than 20 seconds.")
        sys.exit(1)

    download_videos(singer_name, number_of_videos)
    convert_videos_to_audio()
    cut_audio(duration)
    merge_audios(output_filename)


if __name__ == "__main__":
    main()


Overwriting 102313059.py


In [3]:
!python 102313059.py "Arijit Singh" 15 30 mashup.mp3

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
A

In [ ]:
from google.colab import files
files.download("mashup.mp3")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>